In [58]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder.appName('Analyzing the Vocab').getOrCreate())
book = spark.read.text("./1342-0.txt") # Data is read into DataFrame and by default column name is value and type string
display(book)

DataFrame[value: string]

In [6]:
book.printSchema()
book.dtypes

root
 |-- value: string (nullable = true)



[('value', 'string')]

In [8]:
dir(book) #print documentation

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_collect_as_arrow',
 '_ipython_key_completions_',
 '_jcols',
 '_jdf',
 '_jmap',
 '_joinAsOf',
 '_jseq',
 '_lazy_rdd',
 '_repr_html_',
 '_sc',
 '_schema',
 '_session',
 '_sort_cols',
 '_sql_ctx',
 '_support_repr_html',
 'agg',
 'alias',
 'approxQuantile',
 'cache',
 'checkpoint',
 'coalesce',
 'colRegex',
 'collect',
 'columns',
 'corr',
 'count',
 'cov',
 'createGlobalTempView',
 'createOrReplaceGlobalTempView',
 'createOrReplaceTempView',
 'createTempView',
 'crossJoin',
 'crosstab',
 'cube',
 'describe',
 'distinct',
 'drop',
 'dropDuplicates',
 'dropDuplicatesWithinWatermark',
 'drop_du

In [9]:
print(book.__doc__) #print documentation

A distributed collection of data grouped into named columns.

    .. versionadded:: 1.3.0

    .. versionchanged:: 3.4.0
        Supports Spark Connect.

    Examples
    --------
    A :class:`DataFrame` is equivalent to a relational table in Spark SQL,
    and can be created using various functions in :class:`SparkSession`:

    >>> people = spark.createDataFrame([
    ...     {"deptId": 1, "age": 40, "name": "Hyukjin Kwon", "gender": "M", "salary": 50},
    ...     {"deptId": 1, "age": 50, "name": "Takuya Ueshin", "gender": "M", "salary": 100},
    ...     {"deptId": 2, "age": 60, "name": "Xinrong Meng", "gender": "F", "salary": 150},
    ...     {"deptId": 3, "age": 20, "name": "Haejoon Lee", "gender": "M", "salary": 200}
    ... ])

    Once created, it can be manipulated using the various domain-specific-language
    (DSL) functions defined in: :class:`DataFrame`, :class:`Column`.

    To select a column from the :class:`DataFrame`, use the apply method:

    >>> age_col = people

In [11]:
book.show()
book.show(10,truncate = 50)

+--------------------+
|               value|
+--------------------+
|The Project Guten...|
|                    |
|This eBook is for...|
|almost no restric...|
|re-use it under t...|
|with this eBook o...|
|                    |
|                    |
|Title: Pride and ...|
|                    |
| Author: Jane Austen|
|                    |
|Posting Date: Aug...|
|Release Date: Jun...|
|Last Updated: Mar...|
|                    |
|   Language: English|
|                    |
|Character set enc...|
|                    |
+--------------------+
only showing top 20 rows

+--------------------------------------------------+
|                                             value|
+--------------------------------------------------+
|The Project Gutenberg EBook of Pride and Prejud...|
|                                                  |
|This eBook is for the use of anyone anywhere at...|
|almost no restrictions whatsoever.  You may cop...|
|re-use it under the terms of the Project Gutenb...

In [26]:
from pyspark.sql.functions import split,col
lines = book.select(split(book.value," ")
                    .alias("line"))

#Another way to rename
# lines = book.select(split(book.value," "))
# lines = lines.withColumnRenamed("split(value,  , -1)","line")

#Different ways to select columns
# book.select(book.value)
# book.select(book["value"])
# book.select(col("value"))
# book.select("value")

lines.show(5)
lines.printSchema()

+--------------------+
|                line|
+--------------------+
|[The, Project, Gu...|
|                  []|
|[This, eBook, is,...|
|[almost, no, rest...|
|[re-use, it, unde...|
+--------------------+
only showing top 5 rows

root
 |-- line: array (nullable = true)
 |    |-- element: string (containsNull = false)



In [33]:
from pyspark.sql.functions import explode,lower,col
words = lines.select(explode(lines.line).alias("words")) #explode to individual element
lower_word = words.select(lower(col('words')).alias("lower_word")) #lowercase the words

lower_word.show()

+----------+
|lower_word|
+----------+
|       the|
|   project|
| gutenberg|
|     ebook|
|        of|
|     pride|
|       and|
|prejudice,|
|        by|
|      jane|
|    austen|
|          |
|      this|
|     ebook|
|        is|
|       for|
|       the|
|       use|
|        of|
|    anyone|
+----------+
only showing top 20 rows



In [59]:
#Clean the data to just keep alphabets
from pyspark.sql.functions import regexp_extract
clean_data = lower_word.select(regexp_extract(col('lower_word'),'[a-z]+',0).alias('clean_data'))
clean_data.show(10)
print(regexp_extract.__doc__)

+----------+
|clean_data|
+----------+
|       the|
|   project|
| gutenberg|
|     ebook|
|        of|
|     pride|
|       and|
| prejudice|
|        by|
|      jane|
+----------+
only showing top 10 rows

Extract a specific group matched by the Java regex `regexp`, from the specified string column.
    If the regex did not match, or the specified group did not match, an empty string is returned.

    .. versionadded:: 1.5.0

    .. versionchanged:: 3.4.0
        Supports Spark Connect.

    Parameters
    ----------
    str : :class:`~pyspark.sql.Column` or str
        target column to work on.
    pattern : str
        regex pattern to apply.
    idx : int
        matched group id.

    Returns
    -------
    :class:`~pyspark.sql.Column`
        matched value specified by `idx` group id.

    Examples
    --------
    >>> df = spark.createDataFrame([('100-200',)], ['str'])
    >>> df.select(regexp_extract('str', r'(\d+)-(\d+)', 1).alias('d')).collect()
    [Row(d='100')]
    >>> d

In [55]:
# #Regexp Test
# df = spark.createDataFrame([('abc34343aaaabc44545abc',)], ['str'])
# df.select(regexp_extract('str', '[a-z]+', 1).alias('d')).collect()

23/10/11 01:37:00 ERROR Executor: Exception in task 7.0 in stage 27.0 (TID 90)
org.apache.spark.SparkRuntimeException: [INVALID_PARAMETER_VALUE.REGEX_GROUP_INDEX] The value of parameter(s) `idx` in `regexp_extract` is invalid: Expects group index between 0 and 0, but got 1.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.invalidRegexGroupIndexError(QueryExecutionErrors.scala:357)
	at org.apache.spark.sql.catalyst.expressions.RegExpExtractBase$.checkGroupIndex(regexpExpressions.scala:735)
	at org.apache.spark.sql.catalyst.expressions.RegExpExtractBase.checkGroupIndex(regexpExpressions.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenEvaluatorFactory$WholeStageCodegenPartitionEvaluator$$anon$1.hasNext(WholeStageCodegenEvaluatorFactory.scala:43)
	at org.apache.s

Py4JJavaError: An error occurred while calling o452.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 7 in stage 27.0 failed 1 times, most recent failure: Lost task 7.0 in stage 27.0 (TID 90) (172.28.169.28 executor driver): org.apache.spark.SparkRuntimeException: [INVALID_PARAMETER_VALUE.REGEX_GROUP_INDEX] The value of parameter(s) `idx` in `regexp_extract` is invalid: Expects group index between 0 and 0, but got 1.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.invalidRegexGroupIndexError(QueryExecutionErrors.scala:357)
	at org.apache.spark.sql.catalyst.expressions.RegExpExtractBase$.checkGroupIndex(regexpExpressions.scala:735)
	at org.apache.spark.sql.catalyst.expressions.RegExpExtractBase.checkGroupIndex(regexpExpressions.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.jav

In [78]:
words_nonull = clean_data.where(col("clean_data") != "")
#words_nonull.count()

124448

In [83]:
from pyspark.sql.functions import length
#Remove all occurences of "is"
words_withoutis = words_nonull.where(col("clean_data") != 'is')
print(words_withoutis.count())

#Keep only words greater than 3
words_3 = words_withoutis.where(length(col("clean_data")) > 3 )
print(words_3.count())

123564
67645


In [110]:
#GroupBy and Count
words_count = words_3.groupBy("clean_data").count()
print(words_count)

words_count.show()

#words_nonull.select(length(col("clean_data")).alias("length")).groupby("length").count().show()

DataFrame[clean_data: string, count: bigint]
+-------------+-----+
|   clean_data|count|
+-------------+-----+
|       online|    4|
|         some|  209|
|        still|   72|
|         hope|  122|
|        those|   60|
|     cautious|    4|
|    imitation|    1|
|      solaced|    1|
|       poetry|    2|
|    arguments|    5|
| premeditated|    1|
|      elevate|    1|
|       doubts|    2|
|    destitute|    1|
|    solemnity|    5|
|   lieutenant|    1|
|gratification|    1|
|    connected|   14|
|   likelihood|    5|
|        pools|    1|
+-------------+-----+
only showing top 20 rows



In [98]:
#Ordering the results
words_count_ordered = words_count.orderBy('count',ascending=False)
words_count_ordered.show()

+----------+-----+
|clean_data|count|
+----------+-----+
|      that| 1582|
|      with| 1099|
|      have|  846|
| elizabeth|  634|
|      they|  604|
|      were|  566|
|     which|  542|
|     could|  526|
|      been|  515|
|      from|  505|
|      this|  494|
|      very|  486|
|      what|  476|
|     would|  470|
|      your|  464|
|     their|  442|
|      them|  435|
|      will|  418|
|     darcy|  418|
|      said|  401|
+----------+-----+
only showing top 20 rows



In [105]:
#Write to CSV
words_count_ordered.write.csv("./data/simple_count.csv")

In [118]:
#Method Chaining
import pyspark.sql.functions as F 

results = (
            spark.read.text("1342-0.txt")
            .select(F.split(F.col("value")," ").alias("line"))
            .select(F.explode(F.col("line")).alias("word"))
            .select(F.lower(F.col("word")).alias("word"))
            .select(F.regexp_extract(F.col("word"),"[a-z']*",0).alias("word"))
            .where(F.col("word") != "")
            .groupBy("word")
            .count()
            .orderBy(F.col("count"),ascending=False)
)

results.show()

+----+-----+
|word|count|
+----+-----+
| the| 4480|
|  to| 4218|
|  of| 3711|
| and| 3504|
| her| 2199|
|   a| 1982|
|  in| 1909|
| was| 1838|
|   i| 1749|
| she| 1668|
|that| 1487|
|  it| 1482|
| not| 1427|
| you| 1300|
|  he| 1296|
|  be| 1257|
| his| 1247|
|  as| 1174|
| had| 1170|
|with| 1092|
+----+-----+
only showing top 20 rows

